Intro of the author

In [1]:
# This is a Julia version of Willam Nordhaus' DICE2016 model, obtained from
# the model author's website <https://sites.google.com/site/williamdnordhaus/dice-rice>,
# translated from original GAMS code by Lupi,Liuzzi, with the help of 
#Oleg Lugovoy(https://github.com/olugovoy/) and 
# and Libbub (https://github.com/Libbum/DICE.jl)
#Run "install.jl" file to add all required packages:
#
#if Pkg.installed("CSV") == nothing Pkg.add("CSV") end
#if Pkg.installed("JuMP") == nothing Pkg.add("JuMP") end
#if Pkg.installed("Ipopt") == nothing Pkg.add("Ipopt") end
#if Pkg.installed("DataFrames") == nothing Pkg.add("DataFrames") end
#if Pkg.installed("Gadfly") == nothing Pkg.add("Gadfly") end
#if Pkg.installed("JLD") == nothing Pkg.add("JLD") end
#import Pkg
#Pkg.add("JLD2")

Scalars

In [2]:
N = 100 #Number of years to calculate (from 2015 onwards)
tstep = 5 #Years per Period
α = 1.45 #Elasticity of marginal utility of consumption
ρ = 0.015 #Initial rate of social time preference per year ρ
γₑ = 0.3 #Capital elasticity in production function
pop₀ = 7403 #Initial world population 2015 (millions)
popadj = 0.134 #Growth rate to calibrate to 2050 pop projection
popasym = 11100 #Asymptotic population (millions)
δk = 0.1 #Depreciation rate on capital (per year)
q₀ = 105.5 #Initial world gross output 2015 (trill 2010 USD)
k₀ = 223.0 #Initial capital value 2015 (trill 2010 USD)
a₀ = 5.115 #Initial level of total factor productivity
ga₀ = 0.076 #Initial growth rate for TFP per 5 years
δₐ = 0.005 #Decline rate of TFP per 5 years
gσ₁ = -0.0152 #Initial growth of sigma (continuous per year)
δσ = -0.001 #Decline rate of decarbonization per period
eland₀ = 2.6 #Carbon emissions from land 2015 (GtCO2 per year)
deland = 0.115 #Decline rate of land emissions (per period)
e₀ = 35.85 #Industrial emissions 2015 (GtCO2 per year)
μ₀ = 0.03 #Initial emissions control rate for base case 2015
mat₀ = 851.0 #Initial Concentration in atmosphere 2015 (GtC)
mu₀ = 460.0 #Initial Concentration in upper strata 2015 (GtC)
ml₀ = 1740.0 #Initial Concentration in lower strata 2015 (GtC)
mateq = 588.0 #Equilibrium concentration atmosphere  (GtC)
mueq = 360.0 #Equilibrium concentration in upper strata (GtC)
mleq = 1720.0 #Equilibrium concentration in lower strata (GtC)
ϕ₁₂ = 0.12 #Carbon cycle transition matrix coefficient
ϕ₂₃ = 0.007 #Carbon cycle transition matrix coefficient
#t2xco2 = 3.1 #Equilibrium temp impact (oC per doubling CO2)
t2xco2 = 2.5 #Equilibrium temp impact (oC per doubling CO2)
fₑₓ0 = 0.5 #2015 forcings of non-CO2 GHG (Wm-2)
fₑₓ1 = 1.0 #2100 forcings of non-CO2 GHG (Wm-2)
tocean₀ = 0.0068 #Initial lower stratum temp change (C from 1900)
tatm₀ = 0.85 #Initial atmospheric temp change (C from 1900)
ξ₁ = 0.1005 #Climate equation coefficient for upper level
ξ₃ = 0.088 #Transfer coefficient upper to lower stratum
ξ₄ = 0.025 #Transfer coefficient for lower level
η = 3.6813 #Forcings of equilibrium CO2 doubling (Wm-2)
ψ₁₀ = 0.0 #Initial damage intercept
ψ₁ = 0.0 #Damage intercept
#Baseline
#ψ₂ = 0.00236 #Damage quadratic term
ψ₂ = 0.0 #Damage quadratic term
ψ₃ = 2.0 #Damage exponent
θ₂ = 2.6 #Exponent of control cost function
pback = 550.0 #Cost of backstop 2010$ per tCO2 2015
gback = 0.025 #Initial cost decline backstop cost per period
limμ = 1.2 #Upper limit on control rate after 2150
tnopol = 45.0 #Period before which no emissions controls base
cprice₀ = 2.0 #Initial base carbon price (2010$ per tCO2)
gcprice = 0.02 #Growth rate of base carbon price per year
fosslim = 6000.0 #Maximum cumulative extraction fossil fuels (GtC)
scale1 = 0.0302455265681763 #Multiplicative scaling coefficient
scale2 = -10993.704 #Additive scaling coefficient

#High weigth to population -Liddle 2015 [b=1.26, c=1]
v_kappa  = 0  #weight of emissions - varying this parameter gives problem
v_psi  = 0.87  #output elasticity
v_omega = 0.98 #population elasticity
v_epsilon = 1   #total utilitarianism
;

Generating scalars

In [3]:
optlrsav = (δk + .004)/(δk + .004*α + ρ)*γₑ; # Optimal savings rate
ϕ₁₁ = 1 - ϕ₁₂; # Carbon cycle transition matrix coefficient
ϕ₂₁ = ϕ₁₂*mateq/mueq; # Carbon cycle transition matrix coefficient
ϕ₂₂ = 1 - ϕ₂₁ - ϕ₂₃; # Carbon cycle transition matrix coefficient
ϕ₃₂ = ϕ₂₃*mueq/mleq; # Carbon cycle transition matrix coefficient
ϕ₃₃ = 1 - ϕ₃₂; # Carbon cycle transition matrix coefficient
σ₀ = e₀/(q₀*(1-μ₀)); # Carbon intensity 2010 (kgCO2 per output 2010 USD 2010)
λ = η/t2xco2; # Climate model parameter

Parameters

In [4]:
# Backstop price
    pbacktime = Array{Float64}(undef, N); #undef è un inizializzatore
# Growth rate of productivity from 0 to N
    gₐ = Array{Float64}(undef, N);
# Emissions from deforestation
    Etree = Array{Float64}(undef, N);
# Average utility social discount rate
    rr = Array{Float64}(undef, N);
# Carbon price in base case
    cpricebase = Array{Float64}(undef, N);

for i in 1:N
    pbacktime[i] = pback*(1-gback)^(i-1);
    gₐ[i] = ga₀*exp.(-δₐ*5*(i-1));
    Etree[i] = eland₀*(1-deland)^(i-1);
    rr[i] = 1/((1+ρ)^(tstep*(i-1)));
    cpricebase[i] = cprice₀*(1+gcprice)^(5*(i-1));
end

In [5]:
# Level of population and labor
L = Array{Float64}(undef, N);
L[1] = pop₀;
# Level of total factor productivity
A = Array{Float64}(undef, N);
A[1] = a₀;
# Change in sigma (cumulative improvement of energy efficiency)
gσ = Array{Float64}(undef, N);
gσ[1] = gσ₁;
# CO2-equivalent-emissions output ratio
σ = Array{Float64}(undef, N);
σ[1] = σ₀;
# Cumulative from land
cumtree = Array{Float64}(undef, N);
cumtree[1] = 100.0;

for i in 1:N-1
    L[i+1] = L[i]*(popasym/L[i])^popadj;
    A[i+1] = A[i]/(1-gₐ[i]);
    gσ[i+1] = gσ[i]*((1+δσ)^tstep);
    σ[i+1] = σ[i]*exp.(gσ[i]*tstep);
    cumtree[i+1] = cumtree[i]+Etree[i]*(5/3.666);
end

# Adjusted cost for backstop
θ₁ = Array{Float64}(undef, N);
# Exogenous forcing for other greenhouse gases
fₑₓ = Array{Float64}(undef, N);

for i in 1:N
        θ₁[i] = pbacktime[i]*σ[i]/θ₂/1000.0;
#        fₑₓ[i] = if i <= 18
        fₑₓ[i] = if i <= 12
                        #fₑₓ0+(1/17)*(fₑₓ1-fₑₓ0)*(i-1)
                        fₑₓ0+(1/11)*(fₑₓ1-fₑₓ0)*(i-1)
                    else
                        #fₑₓ1
                        fₑₓ1-fₑₓ0
                    end;
end

Let's start with JuMP. The model "dice"

In [6]:
using JuMP, Ipopt

dice = Model(
    with_optimizer(
        Ipopt.Optimizer,
#        linear_solver="ma97",
        linear_solver="MUMPS",
        tol = 0.1,
        print_level = 5,
        print_frequency_iter = 250,
#        start_with_resto = "yes",        
#        expect_infeasible_problem = "yes",
#       max_iter = 5000
        max_iter = 99900
    )
)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Ipopt

Variables

In [7]:
#@variable(dice, 0.0 <= MIU[i=1:N] <= 1.2); # Emission control rate GHGs
@variable(dice, 0.0 <= MIU[i=1:N] <= 1.0); # Emission control rate GHGs
@variable(dice, FORC[1:N]); #Increase in radiative forcing (watts per m2 from 1900)
@variable(dice, 0.0 <= TATM[1:N] <= 12.); #Increase temperature of atmosphere (degrees C from 1900)
@variable(dice, -1.0 <= TOCEAN[1:N] <= 20.0); #Increase temperatureof lower oceans (degrees C from 1900)
@variable(dice, MAT[1:N] >= 10.0);#Carbon concentration increase in atmosphere (GtC from 1750)
@variable(dice, MU[1:N] >= 100.0);#Carbon concentration increase in shallow oceans (GtC from 1750)
@variable(dice, ML[1:N] >= 1000.0);#Carbon concentration increase in lower oceans (GtC from 1750)
@variable(dice, E[1:N]); # Total CO2 emissions (GtCO2 per year)
@variable(dice, EIND[1:N]); #Industrial emissions (GtCO2 per year)
#@variable(dice, EIND[1:N] >= 1.0); #Industrial emissions (GtCO2 per year)
@variable(dice, C[1:N] >= 2.0); #Consumption (trillions 2010 US dollars per year)
@variable(dice, K[1:N] >= 1.0); #Capital stock (trillions 2010 US dollars)
@variable(dice, CPC[1:N] >= 0.01);#Per capita consumption (thousands 2010 USD per year)
#@variable(dice, YPC[1:N] >= 0.01);#Per capita output (thousands 2010 USD per year)
@variable(dice, I[1:N] >= 0.0); # Investment (trillions 2010 USD per year)
@variable(dice, S[1:N]); # Gross savings rate as fraction of gross world product
@variable(dice, RI[1:N]); # Real interest rate (per annum)
@variable(dice, Y[1:N] >= 0.0); # Gross world product net of abatement and damages (trillions 2005
@variable(dice, YGROSS[1:N] >= 0.0); # Gross world product GROSS of abatement and damages (trillions 2010 USD per year)
@variable(dice, YNET[1:N]);# Output net of damages equation (trillions 2005 USD per year)
@variable(dice, DAMAGES[1:N]); # Damages (trillions 2005 USD per year)
@variable(dice, DAMFRAC[1:N]); # Damages as fraction of gross output
@variable(dice, ABATECOST[1:N]); # Cost of emissions reductions  (trillions 2005 USD per year)
@variable(dice, MCABATE[1:N]);# Marginal cost of abatement (2005$ per ton CO2)
@variable(dice, CCA[1:N] <= fosslim); # Cumulative industrial carbon emissions (GTC)
@variable(dice, CCATOT[1:N]);#Total carbon emissions (GtC)
@variable(dice, CPRICE[i=1:N]);# Carbon price (2005$ per ton of CO2)
@variable(dice, PERIODU[1:N]);# One period utility function
@variable(dice, CEMUTOTPER[1:N]);# Period utility
@variable(dice, UTILITY);# Welfare function

Constraints

In [8]:
#Emissions Equation
  # Emissions Equation
@NLconstraint(dice, EEQ[i=1:N], E[i] == EIND[i] + Etree[i]);
  # Industrial Emissions
#@NLconstraint(dice, EINDEQ[i=1:N], EIND[i] == σ[i] * YGROSS[i] * (1-(MIU[i])));
@NLconstraint(dice, EINDEQ[i=1:N], EIND[i] == σ[i] * (1-(MIU[i]))*(YGROSS[i]^v_psi)*(L[i]/1000)^v_omega);
  #Cumulative total carbon emissions
@NLconstraint(dice, CCATOTEQ[i=1:N], CCATOT[i] == CCA[i] + cumtree[i]);
  # Radiative forcing equation
@NLconstraint(dice, FORCE[i=1:N], FORC[i] == η*log(MAT[i]/588.000)/log(2) + fₑₓ[i]);
  # Equation for damage fraction
@NLconstraint(dice, DAMFRACEQ[i=1:N], DAMFRAC[i] == (ψ₁ * TATM[i]) + (ψ₂ * TATM[i]^ψ₃));
  # Damage equation
@NLconstraint(dice, DAMEQ[i=1:N], DAMAGES[i] == YGROSS[i] * DAMFRAC[i]);
  # Cost of emissions reductions equation
@NLconstraint(dice, ABATEEQ[i=1:N], ABATECOST[i] == YGROSS[i] * θ₁[i] * (MIU[i]^θ₂));
  # Equation for MC abatement
@NLconstraint(dice, MCABATEEQ[i=1:N], MCABATE[i] == pbacktime[i] * MIU[i]^(θ₂-1));
  # Carbon price equation from abatement
@NLconstraint(dice, CARBPRICEEQ[i=1:N], CPRICE[i] == pbacktime[i] * (MIU[i]^(θ₂-1)));
#Economy
  # Output gross equation
@NLconstraint(dice, YGROSSEQ[i=1:N], YGROSS[i] == A[i]*((L[i]/1000)^(1-γₑ))*K[i]^γₑ);
  # Output net of damages equation
@NLconstraint(dice, YNETEQ[i=1:N], YNET[i] == YGROSS[i] * (1-DAMFRAC[i]));
  # Output net equation
@NLconstraint(dice, YY[i=1:N], Y[i] == YNET[i] - ABATECOST[i]);
  # Consumption equation
@NLconstraint(dice, CC[i=1:N], C[i] == Y[i] - I[i]);
  #Per capita consumption definition
@NLconstraint(dice, CPCE[i=1:N], CPC[i] == 1000 * C[i] / L[i]);
#Per capita outputn definition
#@NLconstraint(dice, YPCE[i=1:N], YPC[i] == 1000 * Y[i] / L[i]);
  # Savings rate equation
@NLconstraint(dice, SEQ[i=1:N], I[i] == S[i] * Y[i]);
# Instantaneous utility function equation
#@NLconstraint(dice, PERIODUEQ[i=1:N], PERIODU[i] == ((C[i]*1000/L[i])^(1-α)-1)/(1-α)-1);
@NLconstraint(dice, PERIODUEQ[i=1:N], PERIODU[i] == ((((C[i]*1000/L[i])*((σ[i]*(1-(MIU[i]))*(YGROSS[i]^v_psi)*L[i]^v_omega))^v_kappa)^(1-α)-1)/(1-α)-1));
 # Period utility
#@NLconstraint(dice, CEMUTOTPEREQ[i=1:N], CEMUTOTPER[i] == PERIODU[i] * (L[i]) * rr[i]);
@NLconstraint(dice, CEMUTOTPEREQ[i=1:N], CEMUTOTPER[i] == PERIODU[i] * (L[i]^v_epsilon) * rr[i]);

In [9]:
# Dynamic equations
  # Cumulative carbon emissions
@NLconstraint(dice, CCACCA[i=1:N-1], CCA[i+1] == CCA[i] + EIND[i] * 5/3.666);
  # Atmospheric concentration equation
@NLconstraint(dice, MMAT[i=1:N-1], MAT[i+1] == MAT[i]*ϕ₁₁ + MU[i]*ϕ₂₁ + E[i] * 5/3.666);
  # Lower ocean concentration
@NLconstraint(dice, MML[i=1:N-1], ML[i+1] == ML[i]*ϕ₃₃ + MU[i]*ϕ₂₃);
  # Shallow ocean concentration
@NLconstraint(dice, MMU[i=1:N-1], MU[i+1] == MAT[i]*ϕ₁₂ + MU[i]*ϕ₂₂ + ML[i]*ϕ₃₂);
  # Temperature-climate equation for atmosphere
@NLconstraint(dice, TATMEQ[i=1:N-1], TATM[i+1] == TATM[i] +
                ξ₁*((FORC[i+1]-λ*TATM[i])-(ξ₃*(TATM[i]-TOCEAN[i]))));
  # Temperature-climate equation for lower oceans
@NLconstraint(dice, TOCEANEQ[i=1:N-1], TOCEAN[i+1] == TOCEAN[i] + ξ₄*(TATM[i]-TOCEAN[i]));
  # Capital balance equation
@NLconstraint(dice, KK[i=1:N-1], K[i+1] == (1-δk)^tstep * K[i] + tstep * I[i]);
  # Interest rate equation
@NLconstraint(dice, RIEQ[i=1:N-1], RI[i] == (1+ρ)*(CPC[i+1]/CPC[i])^(α/tstep) - 1);

In [10]:
# Savings rate for asympotic equilibrium
#da capire cosa vuol dire. Otteniamo l'ottimo anche senza 
    for i=N-9:N
        JuMP.fix(S[i], optlrsav; force=true);
    end
#Initial conditions
JuMP.fix(CCA[1], 400.; force=true);
JuMP.fix(MAT[1], mat₀; force=true);
JuMP.fix(MU[1], mu₀; force=true);
JuMP.fix(ML[1], ml₀; force=true);
JuMP.fix(TOCEAN[1], tocean₀; force=true);
JuMP.fix(K[1], k₀; force = true); # might be problematic; see LIbbum
JuMP.fix(TATM[1], tatm₀; force=true); # might be problematic; see LIbbum

#To let emi start at e0
#JuMP.fix(EIND[1], e₀ ; force=true); 

Objective Function

In [11]:
# Objective function
@NLobjective(dice, Max, tstep * scale1 * sum(CEMUTOTPER[i] for i=1:N) + scale2);

In [12]:
#BaU: For base run, this subroutine calculates Hotelling rents.
#Carbon price is maximum of Hotelling rent or baseline price

photel = CPRICE;
for i in N
        if i <= tnopol
            JuMP.set_upper_bound(CPRICE[i], max(photel[i],cpricebase[i]));
        end
end

Here we go

In [13]:
optimize!(dice);

dice_status = termination_status(dice)
#println("UTILITY = ", getobjectivevalue(dice), ", $dice_status")


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:     6946
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:     2874

Total number of variables............................:     2684
                     variables with only lower bounds:      896
                variables with lower and upper bounds:      298
                     variables with only upper bounds:       99
Total number of equality constraints.................:     2492
Total number of inequality co

LOCALLY_SOLVED::TerminationStatusCode = 4

In [14]:
#STORE ARRAY.Ex:
using JLD2
UTILITY_B_LIDDLE = JuMP.value.(PERIODU[1:N-80])
WELF_B_LIDDLE = JuMP.value.(CEMUTOTPER[1:N-80])
TEMP_B_LIDDLE = JuMP.value.(TATM[1:N-80])
MIU_B_LIDDLE = JuMP.value.(MIU[1:N-80])
POP_B_LIDDLE = JuMP.value.(L[1:N-80])
E_B_LIDDLE = JuMP.value.(E[1:N-80])
EIND_B_LIDDLE = JuMP.value.(EIND[1:N-80])
YGROSS_B_LIDDLE = JuMP.value.(YGROSS[1:N-80])
YNET_B_LIDDLE = JuMP.value.(YNET[1:N-80])
Y_B_LIDDLE = JuMP.value.(Y[1:N-80])
DAMAGES_B_LIDDLE = JuMP.value.(DAMAGES[1:N-80])
DAMFRAC_B_LIDDLE = JuMP.value.(DAMFRAC[1:N-80])
ABATECOST_B_LIDDLE = JuMP.value.(ABATECOST[1:N-80])
CPRICE_B_LIDDLE = JuMP.value.(CPRICE[1:N-80])
L_B_LIDDLE = JuMP.value.(L[1:N-80])
#YPC_B_HIGHP = JuMP.value.(YPC[1:N-80])
; #without , you see the numbers

In [15]:
#WRITING OUTPUT
@save "output_B_LIDDLE.jld2" UTILITY_B_LIDDLE WELF_B_LIDDLE TEMP_B_LIDDLE MIU_B_LIDDLE POP_B_LIDDLE E_B_LIDDLE EIND_B_LIDDLE YGROSS_B_LIDDLE YNET_B_LIDDLE Y_B_LIDDLE DAMAGES_B_LIDDLE DAMFRAC_B_LIDDLE ABATECOST_B_LIDDLE CPRICE_B_LIDDLE L_B_LIDDLE 

In [16]:
#using Gadfly
#plot(x=1:N-80, y = JuMP.value.(EIND[1:N-80]), Geom.point, Geom.line)

In [17]:
#plot(x=1:N-80, y = JuMP.value.(L[1:N-80]), Geom.point, Geom.line)

In [18]:
#plot(x=1:N-80, y = JuMP.value.(CPRICE[1:N-80]), Geom.point, Geom.line)

In [19]:
CPRICE_B_LIDDLE = JuMP.value.(CPRICE[1:N-80])

20-element Vector{Float64}:
   2.875914591460118
   3.8180116219082594
   5.007812718183196
   6.499126688089606
   8.355395096652897
  10.651181834965286
  13.47385758022193
  16.925493401191883
  21.124977529414682
  26.210368059486434
  32.34149194507415
  39.70279712274199
  48.50646033371453
  58.995749531166176
  71.44864126218569
  86.18170774970892
 103.55433833695304
 123.97348964992884
 147.8994644120591
 175.8538907370662

In [20]:
EIND_B_LIDDLE = JuMP.value.(EIND[1:N-80])

20-element Vector{Float64}:
 137.70787824143892
 155.1781474790077
 171.78433318120395
 187.17412033858963
 201.03748264577268
 213.10804040037436
 223.1612394885939
 231.01042168593455
 236.50171133130078
 239.50844406657154
 239.9256601923498
 237.66500568589157
 232.65023869986524
 224.81342835293032
 214.09184816201724
 200.4254987563026
 183.75512473630835
 164.02049515253114
 141.15855568748094
 115.10077240402359

In [21]:
#YPC_B_HIGHPP = JuMP.value.(YPC[1:N-80])